# ✅ LLM Processor Test Notebook
This notebook is ready for you to paste the full updated `llm_processor_dbconn.py` code.

In [ ]:

# ===================================================================
#   PASTE FULL UPDATED llm_processor_dbconn.py CODE IN THIS CELL
# ===================================================================

print("⚠️ Replace this placeholder with the full updated Python script.")


In [ ]:

# Example of how to run the pipeline once code is pasted above:

# output = run_llm_pipeline(
#     issuer="Manhattan Life",
#     paycode="Default",
#     trandate="2025-11-12",
#     load_task_id="11497",
#     company_issuer_id="3205",
#     csv_path="./inbound/test_raw.csv",
#     template_dir="./carrier_prompts",
#     output_csv_name="manhattan_test_output",
#     server_name="NGCS",
#     database_name="NGCS",
# )
# print("Output file:", output)
